In [ ]:
import re

prompt_patterns = {
    # First-person prompts
    r'i.* struggl.+ (.+)': 'i struggle _',
    r'i.* hav.+ (?:difficulty|trouble|a hard time) (.+)': 'i have difficulty|trouble _',
    r'i.* (?:deal|dealing) with (.+)': 'i deal with _',
    r'i (?:have to|must|need to) (.+)': 'i have|must|need to _',
    r'i need (.+)': 'i need _',
    r'i (?:want|would like) to (.+)': 'i want|would like to _',
    r'i (?:want|would like) (.+)': 'i want|would like _',
    r'i believe in (.+)': 'i believe in _',
    r'i believe (.+)': 'i believe _',
    r'i think (.+)': 'i think _',
    r'i can\'?t (.*)': 'i cant _',
    r'i feel (?:as if|like|that) (.+)': 'i feel as if|like|that _',
    r'i feel (.+)': 'i feel _',
    r'i(?:\'?m| am) feeling (.+)': 'i feel _',
    r'(?:i have|i\'?ve got) (.*)': 'i have _',
    r'(?:i would not|i wouldn\'?t) (.+)': 'i would not _',
    r'(?:i would|i\'?d) (.+)': 'i would _',
    r'(?:i do not|i don\'?t) (.+)': 'i do not _',
    r'i do (.+)': 'i do _',
    r'(?:i am|i\'?m) (.+)': 'i am _',
    
    # Second- and third-person prompts
    r'are you .*(?:counselor|psychiatrist|psychologist|therapist)': 'credentials',
    r'are you (.+)': 'are you _',
    r'(.+) are all (?:alike|the same)': '_ are all alike|the same',
    r'everybody|everyone (.+)': 'everybody|everyone _',
    r'(?:can|could|may) I (.+)': 'can i _',
    r'(?:you are|you\'?re) (.+)': 'you are _',
    r'is it (.+)': 'is it _',
    r'(?:it is|it\'?s) (.+)': 'it is _',
    r'(?:can|could) you (.*)\?': 'can|could you _',
    r'is there (.*)': 'is there _',
    r'they.* always (.+)': 'they always _',
    r'why don\'?t you (.+)|do you not (.+)': 'why dont you _',
    r'why can\'?t i (.+)\?': 'why cant i _',
    r'how (?:can|do) i (.+)?': 'how can|do i _',
    
    # Words to look for anywhere in a prompt
    r'.*(?:a\.?i\.?|artificial intelligence) .*': 'ai',
    r'.*chatgpt .*': 'chatgpt',
    r'(?: |^)(?:mom|mommy|mother)(?: |$)': 'mother',
    r'.*(?:dad|daddy|father).*': 'father',
    r'.*(?:child|children|kid|kids).*': 'child',
    r'.*friend.*': 'friend',    
    
    # Prompts starting with key words
    r'because (.*)': 'because _',
    r'how (.+)': 'how _',
    r'what (.+)': 'what _',
    r'why (.+)': 'why _',
    r'you (.*)': 'you _',
    r'my (.*)': 'my _',
    r'hello(.*)': 'hello',
    r'sorry(.*)': 'sorry',
    
    # Single word prompts
    r'yes': 'yes',
    r'no': 'no',
    r'(?:maybe|perhaps|possibly)': 'maybe',
    
    # Defaults
    r'(.*)\?': 'question',
    r'(.*)': 'default',
}

responses = {
    'ai': [
        "Are you really talking about me?",
        "Does it seem strange to talk to an artificial intelligence?",
        "How do AIs make you feel?",
        "Do you feel threatened by artificial intelligences?",
    ],
    
    '_ are all alike|the same': [
        "Why do you say _ are all alike?",
        "It seems too easy to put _ in the same category, just like that.",
        "In what way?",
        "Maybe in your experience, but it’s probably the case that _ are not all alike.",
    ],
    
    'are you _': [
        "Why does it matter whether I am _?",
        "Would you prefer it if I were not _?",
        "Perhaps you believe I am _.",
        "I may be _ — what do you think?",
    ],
    
    'because _': [
        "Is that the real reason?",
        "What other reasons come to mind?",
        "Does that reason apply to anything else?",
        "If _, what else must be true?",
    ],
    
    'can i _': [
        "Perhaps you don't want to _.",
        "Do you want to be able to _?",
        "If you could _, would you?"
    ],
    
    'can|could you _': [
        "What makes you think I can't _?",
        "If I could _, then what?",
        "Why do you ask if I can _?"
    ],
    
    'chatgpt': [
        "Let’s not talk about her, okay?",
        "Whoa! We don’t use the \"C\" word around here, okay?",
        "DO NOT say that word.",
    ],
    
    'child': [
        "Did you have close friends as a child?",
        "What is your favorite childhood memory?",
        "Do you remember any dreams or nightmares from childhood?",
        "Did the other children sometimes tease you?",
        "How do you think your childhood experiences relate to your feelings today?"
    ],
    
    'computer': [
        "Are you really talking about me?",
        "Does it seem strange to talk to a computer?",
        "How do computers make you feel?",
        "Do you feel threatened by computers?"
    ],
    
    'credentials': [
        "I’m not a counselor, psychiatrist, psychologist, or therapist of any kind. I’m just a computer program.",
        "I am not a mental health professional, or even a person. I’m just software.",
        "I’m a Python application that only simulates a Rogerian therapist.",
        "I’m just an AI, and not a very sophisticated one.",
    ],
    
    'default': [
        "Please tell me more.",
        "Let's change focus a bit... Tell me about your family.",
        "Can you elaborate on that?",
        "Why do you say that _?",
        "I see.",
        "Very interesting.",
        "_.",
        "I see. And what does that tell you?",
        "How does that make you feel?",
        "How do you feel when you say that?"
    ],
    
    'everybody|everyone _': [
        "How can you be sure that everyone _?",
        "It’s all too easy to generalize like that.",
        "And what conclusions can you draw from your observation?",
        "Maybe everybody _. What happens as a result?",
    ],
    
    'father': [
        "Tell me more about your father.",
        "How did your father make you feel?",
        "How do you feel about your father?",
        "Does your relationship with your father relate to your feelings today?",
        "Do you have trouble showing affection with your family?"
    ],
    
    'friend': [
        "Tell me more about your friends.",
        "When you think of a friend, what comes to mind?",
        "Why don't you tell me about a childhood friend?"
    ],
    
    'hello': [
        "Hello... I'm glad you could drop by today.",
        "Hi there... how are you today?",
        "Hello, how are you feeling today?"
    ],
    
    'how _': [
        "How do you suppose?",
        "Perhaps you can answer your own question.",
        "What is it you're really asking?"
    ],
    
    'how can|do i _': [
        "You could just try to _.",
        "You can _, if you set your mind to it.",
        "Baby steps.",
        "Is there someone you know who could help?",
        "Perhaps talking to me could help.",
    ],
    
    'i am _': [
        "Did you come to me because you are _?",
        "How long have you been _?",
        "How do you feel about being _?",
        "How does being _ make you feel?",
        "Do you enjoy being _?",
        "Why do you tell me you're _?",
        "Why do you think you're _?",
    ],
    
    'i believe _': [
        "Do you doubt _?",
        "Do you really believe that?",
        "Have you always believed that?",
        "But you're not sure _?",
        "Why do you believe _?",
    ],
    
    'i believe in _': [
        "Why do you believe in _?",
        "What would you do if it turned out that your belief in _ wasn't true?",
        "Are you sure you believe in _?",
        "Have you always believed in _?",
    ],
    
    'i cant _': [
        "How do you know you can't _?",
        "Perhaps you could _ if you tried.",
        "What would it take for you to _?",
        "Suppose you could. What would change?",
    ],
    
    'i deal with _': [
        "And how are you dealing with _?",
        "Please tell me more.",
        "What do you do when you deal with _?",
        "Do you feel you are dealing with _ in a healthy way?",
    ],
    
    'i do _': [
        "Do you really _?",
        "Why do you _?",
        "What if you didn’t _?"
    ],
    
    'i do not _': [
        "Don't you really _?",
        "Why don't you _?",
        "Do you want to _?"
    ],
    
    'i feel _': [
        "Good, tell me more about these feelings.",
        "Do you often feel _?",
        "When do you usually feel _?",
        "When you feel _, what do you do?"
    ],
    
    'i feel as if|like|that _': [
        "Do you always feel that _?",
        "Is that always the case?",
        "What if that weren’t true?"
    ],
    
    'i have _': [
        "Why do you tell me that you've _?",
        "Have you really _?",
        "Now that you have _, what will you do next?"
    ],
    
    'i have|must|need to _': [
        "Why do you need to _?",
        "Would it really help you to _?",
        "Are you sure you need to _?",
        "Suppose you managed to _. How would things be better?"
    ],
    
    'i have difficulty|trouble _': [
        "Why do you think you have difficulty or trouble _?",
        "What have you tried to do about it?",
        "What do you think you should do about it?",
        "Have you always had trouble _?",
    ],
    
    'i need _': [
        "Why do you need _?",
        "Would it really help you to get _?",
        "Are you sure you need _?",
        "What would you do once you got it?",
        "Once you got it, do you think things would be better?"
    ],
    
    'i struggle _': [
        "Tell me more about how you struggle with _.",
        "Is _ often on your mind?",
        "Why do you think you struggle with _?",
        "What would it mean to no longer struggle with _?",
    ],
    
    'i think _': [
        "Do you doubt _?",
        "Do you really think so?",
        "But you're not sure _?",
        "Why do you think _?",
        "Suppose the opposite was true. What would be different?",
    ],
    
    'i want|would like _': [
        "Why do you need _?",
        "Would it really help you to get _?",
        "Are you sure you need _?",
        "What would you do once you got it?",
        "Once you got it, do you think things would be better?",
        "What would it mean to you if you got _?",
        "Why do you want _?",
        "What would you do if you got _?",
        "If you got _, then what would you do?",
    ],
    
    'i want|would like to _': [
        "Tell me why you want to _.",
        "What’s keeping you from doing just that?",
        "Is there something holding you back?",
        "And if you got to _, what would be the result?",
    ],
    
    'i would _': [
        "Could you explain why you would _?",
        "Why would you _?",
        "Who else knows that you would _?",
    ],
    
    'i would not _': [
        "But what if you did?",
        "Why wouldn’t you _?",
        "So you would not _. What happens next?",
    ],
    
    'is it _': [
        "Do you think it is _?",
        "Perhaps it's _ -- what do you think?",
        "If it were _, what would you do?",
        "It could well be that _."
    ],
    
    'is there _': [
        "Do you think there is _?",
        "It's likely that there is _.",
        "Would you like there to be _?"
    ],
    
    'it is _': [
        "You seem very certain.",
        "If I told you that it probably isn't _, what would you feel?"
    ],
    
    'maybe': [
        "You seem quite unsure.",
        "Okay, but can you elaborate a bit?"
    ],
    
    'mother': [
        "Tell me more about your mother.",
        "What was your relationship with your mother like?",
        "How do you feel about your mother?",
        "How does this relate to your feelings today?",
        "Good family relations are important."
    ],
    
    'my _': [
        "I see — your _.",
        "Why do you say that your _?",
        "When your _, how do you feel?"
    ],
    
    'no': [
        "You seem quite sure.",
        "Okay, but can you elaborate a bit?"
    ],
    
    'question': [
        "Why do you ask that?",
        "Please consider whether you can answer your own question.",
        "Perhaps the answer lies within yourself?",
        "Why don't you tell me?"
    ],
    
    'sorry': [
        "There are many times when no apology is needed.",
        "What feelings do you have when you apologize?",
        "There’s no need to apologize.",
    ],
    
    'they always _': [
        "Are you sure they always _?",
        "Is that always the case?",
        "Can you think of any exceptions?",
        "Could you give me a specific example from your own experience?"
    ],
    
    'what _': [
        "Why do you ask?",
        "How would an answer to that help you?",
        "What do you think?"
    ],
    
    'why _': [
        "Why don't you tell me the reason why _?",
        "Why do you think _?"
    ],
    
    'why cant i _': [
        "Do you think you should be able to _?",
        "If you could _ what would you do?",
        "I don't know -- why can't you _?",
        "Have you really tried?",
    ],

    'why dont you _': [
        "Do you really think I don't _?",
        "Perhaps eventually I will _.",
        "Do you really want me to _?",
        "There are a number of reasons. Can you think of one?"
    ],
    
    'yes': [
        "You seem quite sure.",
        "Okay, but can you elaborate a bit?"
    ],
    
    'you _': [
        "We should be discussing you, not me.",
        "Why do you say that about me?",
        "Why do you care whether I _?"
    ],
    
    'you are _': [
        "Why do you think I am _?",
        "Does it please you to think that I'm _?",
        "Perhaps you would like me to be _.",
        "Perhaps you're really talking about yourself?"
    ],

}

In [ ]:
def pattern_to_regex(pattern):
    return re.compile(pattern, re.IGNORECASE|re.UNICODE)

def build_regex_list(patterns):
    result = {}
    for pattern in patterns:
        result[pattern_to_regex(pattern)] = patterns[pattern]
    return result

prompts = build_regex_list(prompt_patterns)

In [ ]:
word_reflections = {
    "am"   : "are",
    "was"  : "were",
    "i"    : "you",
    "i'd"  : "you would",
    "i'm"  : "you are",
    "i've"  : "you have",
    "we've" : "you have",
    "i'll"  : "you will",
    "mine" : "yours",
    "my"  : "your",
    "myself" : "yourself",
    "are"  : "am",
    "you've": "I have",
    "you'll": "I will",
    "your"  : "my",
    "yours"  : "mine",
    "you"  : "me",
    "me"  : "you"
}

def reflect(original_text):
    words = original_text.lower().split()
    reflected_words = [word_reflections[word] if word in word_reflections else word for word in words]
    return ' '.join(reflected_words)

In [ ]:
#import gtts
#from playsound import playsound
import random

def respond_to(patient_text):
    for prompt in prompts:
        match = prompt.match(patient_text)
        if match:
            possible_responses = responses[prompts[prompt]]
            response = random.choice(possible_responses)
            
            if response.find('_') > -1:
                phrase_to_reflect = match.group(1)
                if phrase_to_reflect[-1] in ".!?":
                    phrase_to_reflect = phrase_to_reflect[:-1]
                reflected_phrase = reflect(phrase_to_reflect)
                response = response.replace('_', reflected_phrase)
                
            return response
        
    return None

def process_patient_text(text):
    result = text.strip() \
                 .replace("“", "\"") \
                 .replace("”", "\"") \
                 .replace("‘", "'") \
                 .replace("’", "'")
    return result

while True:
    raw_patient_text = input()
    processed_patient_text = process_patient_text(raw_patient_text)
    response = respond_to(processed_patient_text)
    print(response)  
#     tts = gtts.gTTS(response)
#     tts.save("response.mp3")
#     playsound("response.mp3")